# **CH01.3. fastText(Subword Skip-gram)**

#### **`Paper Info`** : Enriching Word Vectors with Subword Information
#### $ \hspace{1.75cm} - $ Piotr Bojanowski, Edouard Grave, Armand Joulin, Tomas Mikolov et al. (TACL/2017)

<b></b>

> ## **요약(Summary)**

| Item | Description |
|------|------|
| Research topic | Subword 기반 fastText 임베딩으로 Word-level 모델의 희귀어·OOV 및 형태 정보 손실 문제를 완화하는 방법 연구 |
| Core idea | 각 단어를 문자 n-gram 벡터의 합으로 표현하고 SGNS 목표를 동일하게 최적화하여 파라미터 공유와 OOV 일반화를 달성 |
| Key findings | $ \cdot{} $ 다언어 word similarity·analogy에서 전반적 성능 향상 관찰 <br> $ \cdot{} $ morphologically rich language에서 syntactic analogy 향상 폭이 큼 |
| Contributions | $ \cdot{} $ 간단한 n-gram 합 구조로 형태 정보 통합 <br> $ \cdot{} $ 언어 비의존적이며 사전 형태소 분석기 없이 적용 가능 <br> $ \cdot{} $ 희귀어·OOV에 강건하고 대규모 학습에 실용적 |

<b></b>

> ## **연구 배경(Motivation & Prior Work)**

#### **(1) 기존 연구 및 한계점** :
##### $ \hspace{0.15cm} \cdot{} $ Word2Vec(CBOW·Skip-gram)과 GloVe는 단어를 atomic token으로 다뤄 내부 형태 정보를 놓침
##### $ \hspace{0.15cm} \cdot{} $ 형태소 기반 접근은 언어별 분석기 의존성과 파이프라인 복잡도를 증가시켜 범용 적용이 어려움
##### $ \hspace{0.15cm} \cdot{} $ character-level RNN·CNN·BPE는 문자 조합이나 subword 빈도를 학습하나 단어 표현과의 직접적 합성 규칙이 단순하지 않음
##### $ \hspace{0.15cm} \cdot{} $ 결과적으로 희귀어·OOV에서 의미 표현 품질 저하 및 파라미터 데이터 효율성 저하가 발생

#### **(2) 연구 목표** :
##### $ \hspace{0.15cm} \cdot{} $ 형태가 풍부한 언어에서도 견고한 단어 표현을 제공하여 희귀어·OOV 성능 저하를 최소화
##### $ \hspace{0.15cm} \cdot{} $ 언어·도메인 비의존적으로 대규모 코퍼스에 효율적으로 적용 가능한 간단한 임베딩 방법을 제시
##### $ \hspace{0.15cm} \cdot{} $ word similarity·analogy·language modeling 등 다양한 벤치마크에서 일관된 개선을 보이는 범용 임베딩 확보

#### **(3) 제안된 방법론** :
##### $ \hspace{0.15cm} \cdot{} $ Skip-gram with Negative Sampling의 스코어를 유지하되 입력 벡터를 단어의 문자 n-gram 벡터 합으로 정의하여 파라미터 공유와 OOV 일반화를 동시에 달성

<b></b>

> ## **방법론(Method)**

#### **(1) SGNS 기본 구조** :
##### $ \hspace{0.15cm} \cdot{} $ 중심 단어 $ w_{t} $와 윈도우 내 문맥 $ w_{c} $에 대해 로지스틱 이진 분류 손실을 최소화
##### $ \hspace{0.15cm} \Rightarrow{} \displaystyle{} \mathcal{L} = \sum_{t=1}^{T} \sum_{c \in{} C_{t}} \Big\{ -\log{}\sigma\big(s(w_{t}, w_{c})\big) - \sum_{n \in{} N_{t,c}} \log{}\sigma\big(-s(w_{t}, n)\big) \Big\} $
##### $ \hspace{0.45cm} \text{ where } \, \sigma{}(\cdot{}) : \text{sigmoid} , \; C_{t} : \text{context indices around } t , \; N_{t,c} : \text{negative samples set} $
##### $ \hspace{0.45cm} \text{ and } \, s(w_{i}, w_{o}) = \mathbf{u}_{w_{i}}^{\top} \mathbf{v}_{w_{o}} , \; \mathbf{u}_{w} , \mathbf{v}_{w} \in{} \mathbb{R}^{d} $

#### **(2) Subword 합성 스코어** :
##### $ \hspace{0.15cm} \cdot{} $ 각 단어 $ w $에 경계기호를 포함해 문자 n-gram 집합 $ G_{w} $를 생성하고 각 n-gram에 파라미터 $ \mathbf{z}_{g} \in{} \mathbb{R}^{d} $를 둠
##### $ \hspace{0.15cm} \Rightarrow{} \mathbf{u}_{w} = \displaystyle{} \sum_{g \in{} G_{w}} \mathbf{z}_{g} \;\; \Rightarrow{} \;\; s(w,c) = \mathbf{u}_{w}^{\top} \mathbf{v}_{c} = \sum_{g \in{} G_{w}} \mathbf{z}_{g}^{\top} \mathbf{v}_{c} $
##### $ \hspace{0.45cm} \text{ where } \, G_{w} : \text{character n-gram indices of } w , \; n \in{} \{3,\ldots{},6\} $

##### **(`PLUS`) OOV 표현** : 학습 중 미등장 단어라도 구성 n-gram이 학습되었으면 $ \mathbf{u}_{w} $를 합성 가능

#### **(3) 경사 유도 및 파라미터 공유** :
##### $ \hspace{0.15cm} \cdot{} $ $ s = \mathbf{u}_{w_{t}}^{\top} \mathbf{v}_{w} $일 때 $ -\log{}\sigma(s) $의 미분은 $ \partial L / \partial s = \sigma(-s) $
##### $ \hspace{0.15cm} \Rightarrow{} \partial L / \partial \mathbf{u}_{w_{t}} = \big(1 - \sigma(s)\big) \mathbf{v}_{w} , \;\; \partial L / \partial \mathbf{v}_{w} = \big(1 - \sigma(s)\big) \mathbf{u}_{w_{t}} $
##### $ \hspace{0.15cm} \cdot{} $ $ \mathbf{u}_{w_{t}} = \sum_{g \in{} G_{w_{t}}} \mathbf{z}_{g} $ 이므로 연쇄법칙으로 $ \partial L / \partial \mathbf{z}_{g} = \partial L / \partial \mathbf{u}_{w_{t}} $ for $ g \in{} G_{w_{t}} $
##### $ \hspace{0.15cm} \cdot{} $ 동일 단어 내 모든 n-gram이 동일한 그라디언트를 공유하여 희귀 형태들 간 파라미터 공유가 자연히 발생

#### **(4) 해시 버킷팅으로 메모리 상한화** :
##### $ \hspace{0.15cm} \cdot{} $ n-gram을 FNV-1a 등 해시로 $ K $ 버킷에 사상하고 충돌은 파라미터 공유로 처리
##### $ \hspace{0.15cm} \Rightarrow{} K \approx{} 2 \times 10^{6} \;\; \text{ where } \, K : \text{number of hash buckets controlling memory} $

##### **(`PLUS`) 직관적 효과** : 접사·활용·복합어 등 형태 단위가 벡터에 반영되어 syntactic relation과 희귀어 유사도 개선을 유도

<b></b>

> ## **실험(Experiments)**

#### **(1) 실험 설정** :
##### $ \hspace{0.15cm} \cdot{} $ 데이터셋
| Item | Description |
|------|--------------|
| Training dataset | Wikipedia dumps (9 langs: AR, CS, DE, EN, ES, FR, IT, RO, RU) |
| Data(Sentence;row) size | ? |
| Vocabulary(token) | ? |
| Evaluation dataset | Word similarity (WS353, RW, multi-lingual), Analogy (semantic/syntactic), LM (CS, DE, ES, FR, RU ~1M) |
| Evaluation size | ? |
##### $ \hspace{0.15cm} \cdot{} $ 모델 및 훈련
| Item | Description |
|------|--------------|
| Model type | Skip-gram / Subword Skip-gram (fastText) |
| Hidden layer | None |
| Embedding dimension | 300 |
| Context window size | 1–5 |
| Negatives | 5 |
| Subsampling | 1e-4 |
| Optimizer | SGD + linear decay |
| Epochs | 5 |
| N-gram length | 3–6 |
| Hash buckets | 2e6 |
| Training hardware | ? |
| Training time | ? |
| Baseline models | Skip-gram, CBOW, morphology-based baselines |
| Evaluation metric | Spearman corr., Analogy accuracy, Perplexity |

#### **(2) 분석 및 결과** :
##### $ \hspace{0.15cm} \cdot{} $ word similarity에서 subword 모델이 대부분의 언어와 벤치마크에서 sg·cbow 대비 우수하며 EN WS353만 근소 열위 경향
##### $ \hspace{0.15cm} \cdot{} $ EN RW 등 희귀어 비중이 높은 셋에서 개선 폭이 큼
##### $ \hspace{0.15cm} \cdot{} $ analogy에서는 syntactic task에서 큰 향상이며 semantic task에서는 제한적 이득
##### $ \hspace{0.15cm} \cdot{} $ LSTM language model 초기화에 사용 시 모든 언어에서 perplexity가 sg 초기화보다 낮음
##### $ \hspace{0.15cm} \cdot{} $ 데이터가 매우 작아도 subword는 cbow full-data에 준하거나 상회하는 경향을 보여 데이터 효율성이 높음

##### **(`PLUS`) 효율성** : SGNS 대비 처리량이 약 $ \sim{} 1.5\times $ 느리지만 여전히 대규모 학습에 실용적

#### **(3) 결과 해석** :
##### $ \hspace{0.15cm} \cdot{} $ n-gram 합으로 파라미터가 공유되어 희귀형태에서 분산 추정 분산이 감소하고 OOV에서도 의미 표현이 가능
##### $ \hspace{0.15cm} \cdot{} $ 형태가 풍부한 언어에서 접사·활용·복합어 정보가 임베딩에 반영되어 syntactic relation을 더 잘 포착
##### $ \hspace{0.15cm} \cdot{} $ semantic analogy는 subword 단위만으로는 world knowledge 관계를 충분히 보강하지 못해 이득이 제한적

<b></b>

> ## **결론(Conclusion)**

#### **(1)** Skip-gram의 스코어를 유지한 채 입력을 subword 합으로 치환하는 간단한 설계로 희귀어·OOV 및 형태 정보 손실 문제를 완화

#### **(2)** 언어 비의존적이며 형태소 분석기 없이도 적용 가능해 실무 파이프라인 단순성과 확장성을 제공

#### **(3)** morphologically rich language에서 이득이 특히 크며 LM 초기화 등 다운스트림에서도 일관된 개선을 확인

#### **(4)** 최적 n-gram 길이 선택은 데이터·언어·태스크에 따라 달라지는 열린 문제로 후속 연구의 여지가 큼

<b></b>

> ## **부록(Appendix)**

<b></b>

> ## **비평(Commentary)**

<b></b>

> ## **참고 문헌(Reference)**

#### **(1) 관련 후속 논문** :
##### $ \hspace{0.15cm} \cdot{} $ 

#### **(2) 기타 참고 자료** :
#### - 블로그, 연구실 발표 자료 등등